<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/OpenAI/LangChain_SQL_Queries_Postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation


In [2]:
!pip install openai langchain langchain_experimental langchain-openai -q
!pip install supabase -q
!pip install pyodbc -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Get API Keys and setup

In [3]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain_openai import OpenAI

from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.agent_toolkits import SQLDatabaseToolkit

from supabase import create_client, Client

from sqlalchemy import create_engine

import pandas as pd
import pyodbc

from google.colab import userdata

# setup supabase
supabase_url = userdata.get('SUPABASE_URL')
supabase_key = userdata.get('SUPABASE_KEY')
supabase_client = create_client(supabase_url, supabase_key)

# OpenAI key
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
# Test connection to Supabase using Supabase client

# There are 59 records. Displaying the first 3

response = supabase_client.table('Customer').select("*").execute()

## response object has data attribute that contains the result
## response.data
response_df = pd.DataFrame(response.data)
response_df.head(3)


In [4]:
# Connecting to Supabase using SQL Alchemy

# Make sure the connection string is a proper string within quotes
supabase_conn_str = userdata.get('supabase_conn_str')
#supabase_conn_str = "postgresql+psycopg2://[USERID:PASSWORD]@[HOSTNAME:PORT_NUMBER]/[DATABASE_NAME]"


# Use the connection string with SQLAlchemy's create_engine function
db_engine = create_engine(supabase_conn_str)
db = SQLDatabase(db_engine)

# test the connection
print(db.dialect)
print(db.get_usable_table_names())


postgresql
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track', 'countries']


## Lets run some chains

In [13]:


llm = OpenAI(temperature=0, verbose=True, api_key=openai_api_key)
#llm = OpenAI(api_key=openai_api_key)


In [14]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [16]:
# db_chain.run("How many employees are there?") # Run is being deprecated at 0.2
from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    emp_count = db_chain.invoke("How many employees are there?")
    print(cb)




> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM "Employee"
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.
Tokens Used: 5357
	Prompt Tokens: 5344
	Completion Tokens: 13
Successful Requests: 2
Total Cost (USD): $0.008042


In [17]:
# let's get a description of the playlisttrack table in plain english. Super powerful!
with get_openai_callback() as cb:
    db_chain.invoke("Describe the playlisttrack table")
    print(cb)




> Entering new SQLDatabaseChain chain...
Describe the playlisttrack table
SQLQuery:SELECT "PlaylistId", "TrackId" FROM "PlaylistTrack" LIMIT 5;
SQLResult: [(1, 3402), (1, 3389), (1, 3390), (1, 3391), (1, 3392)]
Answer:The playlisttrack table contains the relationship between playlists and tracks. It has two columns, "PlaylistId" and "TrackId", which are both foreign keys referencing the "Playlist" and "Track" tables respectively. This table allows for many-to-many relationships between playlists and tracks, as a playlist can have multiple tracks and a track can belong to multiple playlists.
> Finished chain.
Tokens Used: 5476
	Prompt Tokens: 5386
	Completion Tokens: 90
Successful Requests: 2
Total Cost (USD): $0.008258999999999999


In [18]:
## I think the LLM got confused here.
## How do you handle that

with get_openai_callback() as cb:
    result = db_chain.run("Give me total sales by genre for all genres")
    print(cb)




> Entering new SQLDatabaseChain chain...
Give me total sales by genre for all genres
SQLQuery:SELECT "Genre"."Name", SUM("InvoiceLine"."UnitPrice" * "InvoiceLine"."Quantity") AS "Total Sales"
FROM "Genre"
INNER JOIN "Track" ON "Genre"."GenreId" = "Track"."GenreId"
INNER JOIN "InvoiceLine" ON "Track"."TrackId" = "InvoiceLine"."TrackId"
GROUP BY "Genre"."Name"
ORDER BY "Total Sales" DESC
LIMIT 5;
SQLResult: [('Rock', Decimal('826.65')), ('Latin', Decimal('382.14')), ('Metal', Decimal('261.36')), ('Alternative & Punk', Decimal('241.56')), ('TV Shows', Decimal('93.53'))]
Answer:The top 5 genres by total sales are Rock, Latin, Metal, Alternative & Punk, and TV Shows.
> Finished chain.
Tokens Used: 5591
	Prompt Tokens: 5479
	Completion Tokens: 112
Successful Requests: 2
Total Cost (USD): $0.0084425


In [19]:
# The answer is false. It still gives only 5. It always sets row count limit to 5 for all queries
# Not sure why it didn't 'pretty print' the SQL code here

with get_openai_callback() as cb:
    result = db_chain.run("Give me total sales in amounts by genre for all genres")
    print(cb)




> Entering new SQLDatabaseChain chain...
Give me total sales in amounts by genre for all genres
SQLQuery:SELECT "Genre"."Name", SUM("InvoiceLine"."UnitPrice" * "InvoiceLine"."Quantity") AS "Total Sales" FROM "Genre" INNER JOIN "Track" ON "Genre"."GenreId" = "Track"."GenreId" INNER JOIN "InvoiceLine" ON "Track"."TrackId" = "InvoiceLine"."TrackId" GROUP BY "Genre"."Name" ORDER BY "Total Sales" DESC LIMIT 5;
SQLResult: [('Rock', Decimal('826.65')), ('Latin', Decimal('382.14')), ('Metal', Decimal('261.36')), ('Alternative & Punk', Decimal('241.56')), ('TV Shows', Decimal('93.53'))]
Answer:The total sales in amounts by genre for all genres are: Rock with $826.65, Latin with $382.14, Metal with $261.36, Alternative & Punk with $241.56, and TV Shows with $93.53.
> Finished chain.
Tokens Used: 5621
	Prompt Tokens: 5482
	Completion Tokens: 139
Successful Requests: 2
Total Cost (USD): $0.008500999999999998


In [20]:
# And yes. The table has sales going into 2025. That is not an error.
# Convert this text-to-speech and we don't need BI Dashboards.

with get_openai_callback() as cb:
    db_chain.run("Give me total sales by quarter")
    print(cb)




> Entering new SQLDatabaseChain chain...
Give me total sales by quarter
SQLQuery:SELECT SUM("Total") AS "Total Sales", DATE_TRUNC('quarter', "InvoiceDate") AS "Quarter" FROM "Invoice" GROUP BY "Quarter" ORDER BY "Quarter" DESC LIMIT 5
SQLResult: [(Decimal('125.86'), datetime.datetime(2025, 10, 1, 0, 0, tzinfo=datetime.timezone.utc)), (Decimal('112.86'), datetime.datetime(2025, 7, 1, 0, 0, tzinfo=datetime.timezone.utc)), (Decimal('108.90'), datetime.datetime(2025, 4, 1, 0, 0, tzinfo=datetime.timezone.utc)), (Decimal('102.96'), datetime.datetime(2025, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)), (Decimal('117.86'), datetime.datetime(2024, 10, 1, 0, 0, tzinfo=datetime.timezone.utc))]
Answer:The total sales by quarter are 125.86 for the fourth quarter of 2025, 112.86 for the third quarter of 2025, 108.90 for the second quarter of 2025, 102.96 for the first quarter of 2025, and 117.86 for the fourth quarter of 2024.
> Finished chain.
Tokens Used: 5645
	Prompt Tokens: 5532
	Completion Token

## Oops!!! Out of tokens :(

In [21]:
with get_openai_callback() as cb:
    db_chain.run("Give me total sales by quarter for each genre")
    print(cb)



> Entering new SQLDatabaseChain chain...
Give me total sales by quarter for each genre
SQLQuery:SELECT
    "Genre"."Name" AS "Genre",
    EXTRACT(QUARTER FROM "Invoice"."InvoiceDate") AS "Quarter",
    SUM("InvoiceLine"."UnitPrice" * "InvoiceLine"."Quantity") AS "Total Sales"
FROM "Genre"
INNER JOIN "Track" ON "Genre"."GenreId" = "Track"."GenreId"
INNER JOIN "InvoiceLine" ON "Track"."TrackId" = "InvoiceLine"."TrackId"
INNER JOIN "Invoice" ON "InvoiceLine"."InvoiceId" = "Invoice"."InvoiceId"
GROUP BY "Genre", "Quarter"
ORDER BY "Genre", "Quarter"
SQLResult: [('Alternative', Decimal('1'), Decimal('5.94')), ('Alternative', Decimal('2'), Decimal('3.96')), ('Alternative', Decimal('4'), Decimal('3.96')), ('Alternative & Punk', Decimal('1'), Decimal('60.39')), ('Alternative & Punk', Decimal('2'), Decimal('60.39')), ('Alternative & Punk', Decimal('3'), Decimal('45.54')), ('Alternative & Punk', Decimal('4'), Decimal('75.24')), ('Blues', Decimal('1'), Decimal('15.84')), ('Blues', Decimal('2'),

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4373 tokens (4117 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

# So what do you guys's think. What do you think is the toughest query. Post it on the Applied AI Slack channel.